## <font color='blue'>SME0104 - Cálculo Numérico</font>

### Primeiro Trabalho em Grupo 
#### Comparação de Métodos na Solução do Laplaciano em Grafos para propagação de informação

**Luis Gustavo Nonato**

------

Considere os arquivos `manh.el` e `manh.xy` que fornecem as arestas e as coordenadas dos vértices do grafo de ruas da ilha de Manhattan, NY (arquivos disponíveis para download no Google Drive).

O grafo de ruas possui diversas componentes conexas, considerando somente a maior componente conexa, você deve realizar as seguintes tarefas:
1. Selecione alguns vértices do grafo $v_{i_1},v_{i_2},\ldots,v_{i_k},\, k<<n$ ($n$ é o número de vértices na maior componente do grafo e $k$ é um número bem menor que $n$, $k=10$ por exemplo) e atribua valores distindos $c_{i_1},c_{i_2},\ldots,c_{i_k}$ a cada um dos vértices selecionados (por exemplo valores no intervalo (0,10]);

---


2. Construa a matriz Laplaciana $L$ do grafo de ruas;
3. Construa a matriz de penalidades $P$, sendo $P$ é uma matriz diagonal onde a entrada $P_{jj}=\alpha$ se $j$ corresponde ao índice de algum dos vértices escolhidos no item 1 acima ($\alpha=1.0e7$ por exemplo), sendo $P_{ii}=0$ caso contrário.
4. Construa um vetor $b$ da seguinte forma:
$$
b_{j} = \left\{\begin{matrix} c_{i_s} & \:{ se } \: j = i_s \\ 0 & \:{ caso \: contrário}\end{matrix}\right.
$$
5. Compare o tempo de solução do sistema
$$
(L+P)x = b
$$
para os métodos:
    - Decomposição LU
    - Cholesky
    - Jacobi e Gaus-Seidel
    - Gradientes Conjugados
    
6. Refaça as tarefas com representação por matriz esparsa e matrizes cheias, comparando os resultados.

Alunos: 
- Kauê Hunnicutt Bazilli - 11212226
- Matheus Vieira Gonçalves - 11200397
- Pedro Henrique dias Junqueira de Souza - 11294312

In [1]:
import sys
import time
%pip install --prefix {sys.prefix} numpy
%pip install --prefix {sys.prefix} scipy
%pip install --prefix {sys.prefix} networkx
%pip install --prefix {sys.prefix} matplotlib

import numpy as np
import scipy as sci
from scipy.sparse import linalg
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import cm

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
# Construção do grafo em memória com base nos arquivos
G = nx.Graph()

#Nós
with open("./manh.xy","r") as file:
    lines = file.readlines()
    for i, line in enumerate(lines):
        [x, y] = map(float, line.split("\t")) #Lê como floats
        G.add_node(i, data=(x, y))

#Arestas
with open("./manh.el","r") as file:
    lines = file.readlines()
    for line in lines:
        [x, y] = map(int, line.split("\t")) #Lê como ints
        G.add_edge(x,y)
        
LCC_generator = max(nx.connected_components(G), key=len)
G = G.subgraph(LCC_generator).copy()

# Desse ponto em diante, G é a maior componente conexa

In [3]:
# 1) Selecione alguns vértices do grafo e atribua valores distintos a cada um dos vértices selecionados

# Uma lista dos índices e outra lista com os seus respectivos valores
c_indexes = np.random.randint(0, len(G), 1000)
c_values = np.random.uniform(0, 10, len(c_indexes))
print(c_indexes)

[ 498 1792 2581 7702 6577 2439 2724 2822 1247 6160 7128 5638 7254 5854
 6456 6995 3508 1730 4898  523 7328 7705 3445 8217 8496  574 2091 1477
 2086 8175 8229 4401 6574 6823 1149 7781 5655 3371 2960 1797 5047 5184
 8457 5086 4821 6434 5169 1642 7159 6242 8608 8657 5677 8556 3124 5335
 5825 5380 1764 1385 6654 4356 2820 2759 6240 3032 6429 6839 2491 1966
 8137 6365 2751 5203 2812 6677 5041 1791 5882 7918 1479 2818  175 7769
 4659 6080  151 1524 2736 7384 4023 6796 6294 6737 2475 1646 3149 2024
 2680 2035  439 7818 1267 6340 6340 7360 4900 7891 2325 4443  796 1055
 2063 5994 7520 7120 2754 3027 1934 1643 1676 3779 4007 1341 8486 4088
 5133 7405 4603 3961  702 7074 6154  231 3624 5306 1879 7355 8619 8423
  393 6810 2990 2692 7300 5681 3026 4941 7360 7012 8683 5579 2696  365
   19 1511 7564 3152  402 8605 5354 7921 8521  610 6228 7590 1877 5645
 4508 3513 7910  404  510 7555 6422 6936 5460 8481 3390  102 3856  934
 1268 7435 3803 6911 6924 6579 3098 4716 4072  727 5822 6098 4414 5277
 4927 

In [4]:
# 2) Construa a matriz Laplaciana L do grafo de ruas

# Obtendo a matriz laplaciana
L = np.array(nx.laplacian_matrix(G).todense())
L.shape

<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


(8708, 8708)

In [5]:
# 3) Construa a matriz de penalidades P

P_diag =  [1e7 if i in c_indexes else 0 for i in range(len(G))]
P = np.diag(P_diag)

In [6]:
# 4) Construa o vetor b 
# Px = b
# b é um vetor com ci nos índices i escolhidos na 1) e 0 em todo o resto.
b = np.zeros(len(G))
for i,ci in enumerate(c_indexes):
    b[ci] = c_values[i]

In [7]:
# TODO: 5) Compare o tempo de solução do sistema (L + P)x = Px = b para os métodos:
#   - Decomposição LU
#   - Cholesky
#   - Jacobi e Gaus-Seidel
#   - Gradientes Conjugados
# NOTE: Pode usar métodos embutidos do numpy e etc...
#
# Lx = 0
# Px = b
#
# Nem P nem L são invertíveis, mas (L + P) é, então resolvemos a soma dos sistemas:
#
# (L + P)x = Px = b

L_plus_P_dense = np.add(L, P)
L_plus_P_csc_sparse =sci.sparse.csc_matrix(L_plus_P_dense)

b_dense = b

print(L_plus_P_csc_sparse.shape)

print(L_plus_P_dense.shape)

print(b_dense.shape)


(8708, 8708)
(8708, 8708)
(8708,)


In [15]:
# Testes de convergência

# Teste da existência da decomposição LU:

# Para a existência da decomposição LU, é necessário que a matriz tenha
# os determinantes dos menores principais não-nulos.
def first_minor_determinants_not_null(m):
    for i in range(1,len(m)+1):
        first_minor_determinant = sci.linalg.det(m[0:i,0:i])
        if first_minor_determinant == 0:
            return False

    return True

# Essa operação é mais custosa do que a decomposição LU em si, devido
# à grande quantidade de determinantes que devem ser calculados. 
# Após o teste da execução da decomposição LU, é possível concluir que ela de fato
# existe, e, como esta é uma condição necessária, pode-se inferir que o teste
# é bem sucedido.
lu_possible = True #first_minor_determinants_not_null(L_plus_P_dense)
print(f"Decomposição LU é possível: {lu_possible}")


# Teste da existência da decomposição de Cholesky:

# Para verificar se é possível realizar a decomposição de Cholesky,
# é necessário verificar se a matriz é simétrica e positiva definida.
def is_symmetric(m):
    #Compara matriz com sua transposta
    return np.allclose(m, m.T) 

# É possível concluir que uma matriz é positiva definida se ela 
# tem autovalores positivos. É possível usar a matriz esparsa aqui 
# para otimizar os cálculos.
def is_pos_def(m):
    # Calcula autovalores
    eigs = sci.sparse.linalg.eigs(m, return_eigenvectors=False)

    for eigenvalue in eigs:
        if eigenvalue <= 0:
            return False
    
    return True


cholesky_possible = is_symmetric(L_plus_P_dense) and is_pos_def(L_plus_P_csc_sparse)
print(f"Decomposição de Cholesky é possível: {cholesky_possible}")


# Teste da convergência do método de Jacobi

# Como condição suficiente para a convergência do método de Jacobi, 
# pode-se usar o critério das linhas ou o critério das colunas, bastando
# um deles ser atendido. Como essas condições são suficientes, e não 
# necessárias, pode ser que o método venha a convergir mesmo com a 
# falha desses critérios. 

# Critério das linhas
def row_criterion(m):
    for i in range(len(m)):
        line_sum = 0
        for j in range(len(m[i])):
            if(i != j):
                line_sum += (abs(m[i][j])/abs(m[i][i]))
        
        if(line_sum >= 1):
            return False

    return True

# Critério das colunas
def column_criterion(m):
    for i in range(len(m)):
        column_sum = 0
        for j in range(len(m[i])):
            if(i != j):
                column_sum += (abs(m[j][i])/abs(m[i][i]))
        
        if(column_sum >= 1):
            return False

    return True

jacobi_guaranteed = row_criterion(L_plus_P_dense) or column_criterion(L_plus_P_dense)
print(f"Método de Jacobi tem convergência garantida: {jacobi_guaranteed}")

# Teste da convergência do método de Gauss-Seidel

# Mais uma vez, temos apenas condições suficientes, e não
# necessárias para a convergência: Deve ser atendido o critério das linhas
# e o critério de Sassenfeld.

def sassenfeld_criterion(m):
    b = [0]
    for j in range(1,len(m)):
        b[0] += abs(m[0][j])
    b[0] /= abs(m[0][0])

    for i in range(1,len(m)):
        b.append(0)
        for j in range(0,i):
            b[i] += abs(m[i][j])*b[j]
        for j in range(i+1,len(m)):
            b[i] += abs(m[i][j])
        b[i] /= abs(m[i][i])

    if(max(b) < 1):
        return True
    else:
        return False

gauss_seidel_guaranteed = row_criterion(L_plus_P_dense) and sassenfeld_criterion(L_plus_P_dense)
print(f"Método de Gauss-seidel tem convergência garantida: {gauss_seidel_guaranteed}")

# Teste de convergência para o método dos gradientes conjugados:

# Para que o método dos gradientes conjugados convirja, temos a condição
# suficiente de que a matriz seja simétrica e positiva definida

conjugate_gradient_guaranteed = is_pos_def(L_plus_P_csc_sparse) and is_symmetric(L_plus_P_dense)
print(f"Método dos gradientes conjugados tem convergência garantida: {conjugate_gradient_guaranteed}")


Decomposição LU é possível: True
Decomposição de Cholesky é possível: True
Método de Jacobi tem convergência garantida: False
Método de Gauss-seidel tem convergência garantida: False
Método dos gradientes conjugados tem convergência garantida: True


In [30]:
# Decomposição LU

def dense_lu_solver(A,b):
	lu, piv = sci.linalg.lu_factor(A)
	x = sci.linalg.lu_solve((lu,piv),b)
	return x

def sparse_lu_solver(A,b):
	lu = linalg.splu(A)
	x = lu.solve(b)
	return x


In [9]:
# Decomposição de cholesky

def dense_cholesky_solver(A,b):
	c, low = sci.linalg.cho_factor(A)
	x = sci.linalg.cho_solve((c,low),b)
	return x

def sparse_cholesky_solver(A,b):
	c, low = sci.linalg.cho_factor(A)
	x = sci.linalg.cho_solve((c,low),b)
	return False


In [10]:
# gradientes conjugados

def sparse_cg_solver(A, b):
    return linalg.cg(A, b, tol=1e-12)


def dense_cg_solver(A, b):
    return linalg.cg(A, b, tol=1e-12)



In [11]:
# Gauss-jacobi

def dense_jacobi_solver(A, b, err, err_relative):
    count = 0
    r, c = A.shape
    x = np.zeros(r)

    D = np.diag(A)
    R = A - np.diagflat(D)
    curr_err = err + 1
    curr_err_relative = err_relative + 1

    while curr_err > err or curr_err_relative > err_relative:
        last_x = x.copy()
        x = (b - np.dot(R, x)) / D

        max_distance = -1
        max_module = -1

        for i in range(c):
            curr_distance = abs(last_x[i] - x[i])
            max_distance = curr_distance if curr_distance > max_distance else max_distance
            max_module = abs(x[i]) if abs(x[i]) > max_module else max_module

        curr_err = max_distance
        curr_err_relative = max_distance/max_module

    return x




In [12]:
# Gauss-Seidel
def dense_gauss_seidel_solver(A, b, err, err_relative):
    r, c = A.shape
    x = np.zeros(r)
    count = 0
    curr_err = err + 1
    curr_err_relative = err_relative + 1

    while curr_err > err or curr_err_relative > err_relative:

        last_x = x.copy()

        for i in range(r):
            x[i] = (b[i] - np.dot(A[i, :i], x[:i]) -
                    np.dot(A[i, (i+1):], last_x[(i+1):])) / A[i, i]

        max_distance = -1
        max_module = -1

        for i in range(c):
            curr_distance = abs(last_x[i] - x[i])
            max_distance = curr_distance if curr_distance > max_distance else max_distance
            max_module = abs(x[i]) if abs(x[i]) > max_module else max_module

        curr_err = max_distance
        curr_err_relative = max_distance/max_module

    return x



In [13]:
def execute_method(method, *args):
    start = time.time()
    answer = method(*args)
    total_time = time.time()-start
    return answer, total_time


In [14]:
std_dense_solution, std_dense_solution_time = execute_method(
    np.linalg.solve, L_plus_P_dense, b_dense)

print(std_dense_solution)
print(f"tempo do método padrão np.linalg.solve: {std_dense_solution_time}")

[6.46099573e-07 4.51169052e-07 4.82258249e-07 ... 6.38453532e-07
 5.92435793e-07 4.15223484e-07]
tempo do método padrão np.linalg.solve: 7.4248127937316895


In [15]:
std_sparse_solution, std_sparse_solution_time = execute_method(
    linalg.spsolve, L_plus_P_csc_sparse, b_dense)

print(std_sparse_solution)
print(
    f"tempo do método padrão scipy.sparse.linalg.solve: {std_sparse_solution_time}")

[6.46099573e-07 4.51169052e-07 4.82258249e-07 ... 6.38453532e-07
 5.92435793e-07 4.15223484e-07]
tempo do método padrão scipy.sparse.linalg.solve: 0.018331289291381836


In [16]:
dense_lu_solution, dense_lu_solution_time = execute_method(
    dense_lu_solver, L_plus_P_dense, b_dense)

print(dense_lu_solution)
print(
    f"tempo do método de decomposição LU em uma matriz densa: {dense_lu_solution_time}")

[6.46099573e-07 4.51169052e-07 4.82258249e-07 ... 6.38453532e-07
 5.92435793e-07 4.15223484e-07]
tempo do método de decomposição LU em uma matriz densa: 8.300081729888916


In [17]:

sparse_lu_solution, sparse_lu_solution_time = execute_method(
    sparse_lu_solver, L_plus_P_csc_sparse, b_dense)

print(sparse_lu_solution)
print(
    f"tempo do método de decomposição LU em uma matriz esparsa: {sparse_lu_solution_time}")

[6.46099573e-07 4.51169052e-07 4.82258249e-07 ... 6.38453532e-07
 5.92435793e-07 4.15223484e-07]
tempo do método de decomposição LU em uma matriz esparsa: 0.027009248733520508


In [18]:
dense_cholesky_solution, dense_cholesky_solution_time = execute_method(
    dense_cholesky_solver, L_plus_P_dense, b_dense)

print(dense_cholesky_solution)
print(
    f"tempo do método de decomposição cholesky em uma matriz densa: {dense_cholesky_solution_time}")

[6.46099573e-07 4.51169052e-07 4.82258249e-07 ... 6.38453532e-07
 5.92435793e-07 4.15223484e-07]
tempo do método de decomposição cholesky em uma matriz densa: 5.050318717956543


In [19]:

dense_cg_solution, dense_cg_solution_time = execute_method(
    dense_cg_solver, L_plus_P_dense, b_dense)

print(dense_cg_solution)
print(
    f"tempo do método de gradiente conjugados em uma matriz densa: {dense_cg_solution_time}")

(array([6.46089357e-07, 4.51168726e-07, 4.82259740e-07, ...,
       6.38450429e-07, 5.92428616e-07, 4.15224376e-07]), 0)
tempo do método de gradiente conjugados em uma matriz densa: 6.937185049057007


In [20]:
sparse_cg_solution, sparse_cg_solution_time = execute_method(
    sparse_cg_solver, L_plus_P_csc_sparse, b_dense)

print(sparse_cg_solution)
print(
    f"tempo do método de gradiente conjugados em uma matriz esparsa: {sparse_cg_solution_time}")

(array([6.46089357e-07, 4.51168726e-07, 4.82259740e-07, ...,
       6.38450429e-07, 5.92428616e-07, 4.15224376e-07]), 0)
tempo do método de gradiente conjugados em uma matriz esparsa: 0.036994218826293945


In [21]:
dense_jacobi_solution, dense_jacobi_solution_time = execute_method(
    dense_jacobi_solver, L_plus_P_dense, b_dense, 1e-12, 1e-12)

print(dense_jacobi_solution)
print(
    f"tempo do método de Jacobi em uma matriz densa: {dense_jacobi_solution_time}")

[6.46099573e-07 4.51169052e-07 4.82258249e-07 ... 6.38453532e-07
 5.92435793e-07 4.15223484e-07]
tempo do método de Jacobi em uma matriz densa: 133.0290231704712


In [22]:
dense_gauss_seidel_solution, dense_gauss_seidel_solution_time = execute_method(dense_gauss_seidel_solver,
    L_plus_P_dense, b_dense, 1e-12, 1e-12)

print(dense_gauss_seidel_solution)
print(
    f"tempo do método de Seidel em uma matriz densa: {dense_gauss_seidel_solution_time}")

[6.46099573e-07 4.51169052e-07 4.82258249e-07 ... 6.38453532e-07
 5.92435793e-07 4.15223484e-07]
tempo do método de Seidel em uma matriz densa: 220.86482334136963
